In [2]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

In [3]:
from python.plugins import addFiles
# files = ["coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2016.pkl"]
#files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2016.pkl"]
# files = ["coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2018.pkl", "coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2016.pkl", "coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2017.pkl"]
# files = ["coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2017.pkl", "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2018.pkl", "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2016.pkl"]
# files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2016.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2018.pkl"]
files = ["coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2016.pkl", "coffeaOutput/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2017.pkl", "coffeaOutput/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2018.pkl"]
# combinedFile = addFiles(files)
# print(combinedFile)




In [5]:
%matplotlib inline
# fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecALL.pkl"
# fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjecALL.pkl"
fname = "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_Nonejesjec2016APV.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
axis_names = [ax.name for ax in result['jet_pt_mass_reco_u'].axes]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
print(cats)
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
if "QCD" in fname:
    for syst in cats:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_mreco_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","mreco")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        response_matrix_g_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges= result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                           'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
    rootfile['response']=response_entries
else:
    for syst in cats:
        rootfile['ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'syst':syst}].project("ptreco","mreco")
rootfile.close()

2016
2016
['dataset', 'syst', 'ptreco', 'mreco']
['nominal']


In [ ]:
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjecALL.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result.keys())
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
print(response_matrix_g_values.shape)
response_matrix_g_values= result['response_matrix_g'][{'syst':'nominal'}].project("ptreco", "mreco", "ptgen", "mgen").values()
nptreco,nmassreco,nptgen,nmassgen = response_matrix_g_values.shape
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2) ) 
print(response_matrix_g_final.shape)
response_matrix_g_final2 = result['response_matrix_g'][{'syst':"nominal"}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True).reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(response_matrix_g_final[26,:]))
print(np.sum(response_matrix_g_final2[26,:]))
print(np.sum(response_matrix_u_final[26,:]))

In [ ]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [ ]:
# print(fname["Events"].keys("*HTXS*"))

In [ ]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists.keys())
hist_vals_g, hist_edges, hist_widths = hists['ptreco_mreco_g_nominal'].to_numpy(flow=True)
hist_vals_u, hist_edges, hist_widths = hists['ptreco_mreco_u_nominal'].to_numpy(flow=True)
print(hist_vals_g[1][0])
print(hist_vals_u[1][0])

In [ ]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists['response'].keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0].reshape((nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(resp_groomed[26,:]))